# 2. Modeling

This worksheet focuses on training models that predict the outcomes of the loans.

In [11]:
import pandas as pd
import numpy as np
from joblib import dump

np.random.seed(1151)

## Preparing data for modeling

In order to start training models we need to choose which features to use. After that we check for null values and deal with categorical values by one-hot encoding them. The last step is to create the input and target values.

In [12]:
df = pd.read_csv("data/train_data.csv")

# Columns that are used as inputs in the models.
input_cols = ['NewCreditCustomer', 'VerificationType', 'LanguageCode', 'Age', 'Gender',
                 'Amount', 'Interest', 'LoanDuration', 'MonthlyPayment',
                 'Education', 'EmploymentDurationCurrentEmployer', 'HomeOwnershipType', 'IncomeTotal',
                 'ExistingLiabilities', 'LiabilitiesTotal', 'Rating',
                 'CreditScoreEeMini', 'NoOfPreviousLoansBeforeLoan', 'AmountOfPreviousLoansBeforeLoan',
                 'PreviousRepaymentsBeforeLoan', 'PreviousEarlyRepaymentsBefoleLoan', 'PreferLoan']

df = df[input_cols]

In [13]:
# Check for null values.
df.isna().sum()

# If df contains null values, they should be dropped.
# df.dropna(inplace=True)

NewCreditCustomer                    0
VerificationType                     0
LanguageCode                         0
Age                                  0
Gender                               0
Amount                               0
Interest                             0
LoanDuration                         0
MonthlyPayment                       0
Education                            0
EmploymentDurationCurrentEmployer    0
HomeOwnershipType                    0
IncomeTotal                          0
ExistingLiabilities                  0
LiabilitiesTotal                     0
Rating                               0
CreditScoreEeMini                    0
NoOfPreviousLoansBeforeLoan          0
AmountOfPreviousLoansBeforeLoan      0
PreviousRepaymentsBeforeLoan         0
PreviousEarlyRepaymentsBefoleLoan    0
PreferLoan                           0
dtype: int64

In [14]:
# One-hot encoding.
df = pd.get_dummies(df)
df

,NewCreditCustomer,VerificationType,LanguageCode,Age,Gender,Amount,Interest,LoanDuration,MonthlyPayment,Education,...,EmploymentDurationCurrentEmployer_UpTo4Years,EmploymentDurationCurrentEmployer_UpTo5Years,Rating_A,Rating_AA,Rating_B,Rating_C,Rating_D,Rating_E,Rating_F,Rating_HR
0,True,1.0,3,24,1.0,6911.0,29.53,60,230.77,4.0,...,0,0,0,0,0,0,1,0,0,0
1,True,4.0,3,18,1.0,531.0,18.07,60,14.89,3.0,...,0,0,0,0,1,0,0,0,0,0
2,False,4.0,1,40,0.0,531.0,36.43,60,19.43,5.0,...,0,1,0,0,0,0,1,0,0,0
3,True,4.0,1,37,0.0,4253.0,31.75,36,189.96,1.0,...,0,0,0,0,0,1,0,0,0,0
4,False,4.0,1,44,0.0,5525.0,24.73,60,179.70,4.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42332,True,4.0,1,53,0.0,530.0,20.80,36,21.68,4.0,...,0,1,0,0,0,1,0,0,0,0
42333,False,4.0,1,61,0.0,2550.0,11.52,60,64.61,4.0,...,0,0,1,0,0,0,0,0,0,0
42334,False,1.0,1,26,1.0,1169.0,9.90,36,41.34,5.0,...,0,1,1,0,0,0,0,0,0,0
42335,True,4.0,1,45,0.0,3720.0,33.28,36,177.07,1.0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
# Defining the input and target values.
X = df.drop('PreferLoan', axis=1)
y = df['PreferLoan']

An imbalance of preferred loans and not preferred loans can be seen, but it should not be a problem.

In [16]:
y.value_counts()

0    32286
1    10051
Name: PreferLoan, dtype: int64

## Training the models

We create a dictionary that consists of different models and hyperparameters for those models. After that we use RandomizedSearchCV for finding the optimal hyperparameters. In this instance we prefer it to GridSearchCV because it is a lot faster. However with GridSearchCV it is possible to do a more exhaustive search and therefore potentially get more accurate hyperparameters.

We create a dictionary of 3 different models that are commonly used for such classification tasks. Parameters with different possible values are also included.

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

scale_pos_weight = y.value_counts()[1]/y.value_counts()[0]

model_params = {
    'random_forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
            'max_depth': [50, 75, 100, 125, 150, 175, 200, None],
            'max_features': ['auto', 'sqrt', 'log2', 1, 2, 3, 4, 5, 6],
            'min_samples_leaf': [1, 2, 3, 4, 8, 10, 12, 15],
            'min_samples_split' : [1, 2, 3, 4, 8, 10, 12, 15],
            'n_estimators': [10, 50, 75, 100, 125, 150, 175, 200],
            'criterion': ['gini', 'entropy']
        }
    },
    'logistic_regression' : {
        'model' : LogisticRegression(),
        'params' : {
            'penalty': ['none', 'l1', 'l2', 'elasticnet'],
            'solver' : ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
            'C' : [0.01, 0.05, 0.1, 0.25, 0.5, 0,75, 1, 1.25, 2, 5, 10],
            'max_iter' : [10, 50, 75, 100, 150, 200, 250, 300]
        }
    },
    'xgboost' : {
        'model' : xgb.XGBClassifier(),
        'params' : {
            "max_depth": [1, 2, 3, 4, 5, 7],
            "gamma": [0, 0.1, 0.25, 0.5, 1],
            "reg_lambda": [0, 1, 5, 10, 15, 25, 50],
            "subsample": [0.7, 0.8, 0.9],
            "colsample_bytree": [0.1, 0.25, 0.5, 0.75, 1],
            'scale_pos_weight ' : [scale_pos_weight, 1, 3, 5, 10, 15, 20]
        }
    }
}

We create an empty list "scores" in which we append the training results and best parameters. We also use three different scoring functions to train the models as it is interesting to see how models with different scoring functions perform in the validation step. After that models with the best parameters are saved so they can be used in the validation step.

In [18]:
# List for keeping track of training results.
scores = []

# List of scoring functions.
score_func = ['precision', 'f1', 'roc_auc']

In [ ]:
for s in score_func:
    for m_name, m_params in model_params.items():
        model_name = m_name + "_" + s

        clf = RandomizedSearchCV(m_params['model'], m_params['params'], cv = 3, return_train_score = False, scoring = s, n_iter=25, n_jobs=-1)
        clf.fit(X, y)
        scores.append({
            'model': model_name,
            'best_score': clf.best_score_,
            'best_params': clf.best_params_
        })
        dump(clf, f'models/{model_name}.joblib')

The results can be seen below. The mean cross-validated score of the best model can be seen for each model and scoring function. It is hard to draw any conclusions from these results because each scoring function is calculated differently. And we have yet to seen how these models perform when using different threshold levels for the classification probability.

In [20]:
pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

,model,best_score,best_params
0,random_forest_precision,0.764299,"{'n_estimators': 100, 'min_samples_split': 3, ..."
1,logistic_regression_precision,0.640440,"{'solver': 'liblinear', 'penalty': 'l1', 'max_..."
2,xgboost_precision,0.641325,"{'subsample': 0.7, 'scale_pos_weight ': 10, 'r..."
3,random_forest_f1,0.317379,"{'n_estimators': 10, 'min_samples_split': 3, '..."
4,logistic_regression_f1,0.194690,"{'solver': 'newton-cg', 'penalty': 'l2', 'max_..."
5,xgboost_f1,0.363161,"{'subsample': 0.8, 'scale_pos_weight ': 10, 'r..."
6,random_forest_roc_auc,0.739559,"{'n_estimators': 175, 'min_samples_split': 4, ..."
7,logistic_regression_roc_auc,0.687908,"{'solver': 'newton-cg', 'penalty': 'l2', 'max_..."
8,xgboost_roc_auc,0.737722,"{'subsample': 0.8, 'scale_pos_weight ': 20, 'r..."
